# Imports

In [1]:
import datetime
import os
import sys
#from dataclasses import dataclass
#import sklearn
from sklearn import model_selection
import matplotlib
from sklearn.model_selection import TimeSeriesSplit, train_test_split
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import tensorflow as tf
#from tensorflow import keras
import statsmodels.api as sm
#from pandas_profiling import ProfileReport
from plotly import express as px
import plotly.graph_objects as go
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn
import tqdm
from tqdm import tqdm
pd.options.plotting.backend = "plotly"


# Set the current working directory and get paths of load profiles

In [2]:
def get_directories(path) -> list:
    """
    Returns a list of directories in the given path.
    """
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
directories = get_directories('Profiles')
directories.sort()

# Builds load profile dataframe using all Excel Files


In [3]:

directories = get_directories('Profiles')
load_data = pd.DataFrame()
for directory in directories:
    file_name = directory.replace("_", ' ')
    #profile_df = load_profile_df('Profiles/ERCOT_Backcasted_Load_Profiles_2016/ERCOT Backcasted Load Profiles 2016.xls')
    try: 
        profile_df = pd.read_excel(f'Profiles/{directory}/{file_name}.xls', index_col = 0, engine='xlrd')
    except:
        profile_df = pd.read_excel(f'Profiles/{directory}/{file_name}.xlsx', index_col = 0, engine = 'openpyxl')
    #prof = ProfileReport(profile_df, title=file_name, minimal=True)
    load_data = pd.concat([load_data, profile_df])
    #prof.to_file(f'Profiles/{directory}/{file_name}.html')
    #profile_df["Date"] = pd.to_datetime(profile_df["Date"])
save_load_df = load_data.copy()
    #date = profile_df["Date"]
    #profile_df.set_index("Date", inplace=True)
    #print(profile_df.columns)
    #profile_df["PType_WZ"] = profile_df["PType_WZ"].astype(str)
#profile_df.plot()
#plt.savefig(f'Profiles/{directory}/{directory}_plot.png')
#plt.show()
save_load_df.reset_index(inplace=True)

/home/austin/anaconda3/envs/bittensor/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
if 'PType_WZ' in save_load_df.columns:
    save_load_df.rename(columns={'PType_WZ':'Station'}, inplace=True)

save_load_df.head()

,Station,Date,int_kWh1,int_kWh2,int_kWh3,int_kWh4,int_kWh5,int_kWh6,int_kWh7,int_kWh8,...,int_kWh92,int_kWh93,int_kWh94,int_kWh95,int_kWh96,int_kWh97,int_kWh98,int_kWh99,int_kWh100,ADDTIME
0,BUSHIDG_COAST,2019-01-01,11.406,11.329,11.279,11.236,11.219,11.211,11.174,11.167,...,12.022,11.810,11.708,11.609,11.462,NaN,NaN,NaN,NaN,2019-01-05
1,BUSHIDG_COAST,2019-01-02,11.530,11.461,11.422,11.398,11.407,11.416,11.391,11.405,...,12.062,11.841,11.728,11.623,11.470,NaN,NaN,NaN,NaN,2019-01-06
2,BUSHIDG_COAST,2019-01-03,11.617,11.546,11.505,11.477,11.483,11.489,11.461,11.470,...,11.950,11.748,11.645,11.546,11.399,NaN,NaN,NaN,NaN,2019-01-07
3,BUSHIDG_COAST,2019-01-04,11.374,11.301,11.255,11.220,11.218,11.215,11.179,11.172,...,12.381,12.167,12.067,11.969,11.821,NaN,NaN,NaN,NaN,2019-01-08
4,BUSHIDG_COAST,2019-01-05,11.652,11.568,11.509,11.453,11.409,11.392,11.355,11.354,...,12.338,12.133,12.044,11.952,11.811,NaN,NaN,NaN,NaN,2019-01-09


In [5]:
if "ADDTIME" in save_load_df.columns:
    save_load_df.drop(columns=["ADDTIME"], inplace=True)
id_vars = ['Station', 'Date']

load_columns = save_load_df.columns.tolist()[2:98]
daily_time_intervals = pd.Series(pd.date_range('1/1/2020', '1/2/2020', freq='15min', closed='left')).dt.time # Dummy dates used to get dail time intervals

new_load_columns = {load_columns[i]: daily_time_intervals[i] for i in range(len(load_columns))}

save_load_df.rename(columns=new_load_columns, inplace=True)
load_columns = save_load_df.columns.tolist()[2:98]
all_columns = save_load_df.columns.tolist()[1:99]

dates = save_load_df['Date']
save_load_df.head()

,Station,Date,00:00:00,00:15:00,00:30:00,00:45:00,01:00:00,01:15:00,01:30:00,01:45:00,...,22:30:00,22:45:00,23:00:00,23:15:00,23:30:00,23:45:00,int_kWh97,int_kWh98,int_kWh99,int_kWh100
0,BUSHIDG_COAST,2019-01-01,11.406,11.329,11.279,11.236,11.219,11.211,11.174,11.167,...,12.157,12.022,11.810,11.708,11.609,11.462,NaN,NaN,NaN,NaN
1,BUSHIDG_COAST,2019-01-02,11.530,11.461,11.422,11.398,11.407,11.416,11.391,11.405,...,12.212,12.062,11.841,11.728,11.623,11.470,NaN,NaN,NaN,NaN
2,BUSHIDG_COAST,2019-01-03,11.617,11.546,11.505,11.477,11.483,11.489,11.461,11.470,...,12.090,11.950,11.748,11.645,11.546,11.399,NaN,NaN,NaN,NaN
3,BUSHIDG_COAST,2019-01-04,11.374,11.301,11.255,11.220,11.218,11.215,11.179,11.172,...,12.511,12.381,12.167,12.067,11.969,11.821,NaN,NaN,NaN,NaN
4,BUSHIDG_COAST,2019-01-05,11.652,11.568,11.509,11.453,11.409,11.392,11.355,11.354,...,12.452,12.338,12.133,12.044,11.952,11.811,NaN,NaN,NaN,NaN


In [6]:
sorted_load = save_load_df.sort_values(by=['Station', 'Date']).reset_index(drop=True)
sorted_load.head()

,Station,Date,00:00:00,00:15:00,00:30:00,00:45:00,01:00:00,01:15:00,01:30:00,01:45:00,...,22:30:00,22:45:00,23:00:00,23:15:00,23:30:00,23:45:00,int_kWh97,int_kWh98,int_kWh99,int_kWh100
0,BUSHIDG_COAST,2016-01-01,11.489,11.413,11.364,11.325,11.312,11.307,11.273,11.273,...,12.015,11.900,11.711,11.624,11.536,11.399,NaN,NaN,NaN,NaN
1,BUSHIDG_COAST,2016-01-02,11.668,11.589,11.538,11.494,11.472,11.464,11.430,11.431,...,12.307,12.199,12.007,11.923,11.836,11.699,NaN,NaN,NaN,NaN
2,BUSHIDG_COAST,2016-01-03,11.457,11.381,11.332,11.291,11.271,11.266,11.235,11.241,...,12.075,11.953,11.746,11.651,11.558,11.416,NaN,NaN,NaN,NaN
3,BUSHIDG_COAST,2016-01-04,11.358,11.280,11.228,11.182,11.162,11.150,11.111,11.100,...,12.145,11.996,11.804,11.699,11.598,11.450,NaN,NaN,NaN,NaN
4,BUSHIDG_COAST,2016-01-05,11.492,11.418,11.373,11.338,11.336,11.334,11.300,11.296,...,12.350,12.210,12.008,11.906,11.806,11.658,NaN,NaN,NaN,NaN


In [7]:
#sorted_load = sorted_load.transpose()

In [8]:
#sorted_load.head()


# Plots a single day and also prints first 5 rows, shape, columns, and index of the load profile dataframe

In [9]:
if 'int_kWh97' in save_load_df.columns:
    try:
        sorted_load = sorted_load.drop(columns=['int_kWh97', 'int_kWh98', 'int_kWh99', 'int_kWh100'])
    except:
        pass
sorted_load = sorted_load.fillna(0)
sorted_load.head()


,Station,Date,00:00:00,00:15:00,00:30:00,00:45:00,01:00:00,01:15:00,01:30:00,01:45:00,...,21:30:00,21:45:00,22:00:00,22:15:00,22:30:00,22:45:00,23:00:00,23:15:00,23:30:00,23:45:00
0,BUSHIDG_COAST,2016-01-01,11.489,11.413,11.364,11.325,11.312,11.307,11.273,11.273,...,12.304,12.261,12.177,12.112,12.015,11.900,11.711,11.624,11.536,11.399
1,BUSHIDG_COAST,2016-01-02,11.668,11.589,11.538,11.494,11.472,11.464,11.430,11.431,...,12.539,12.505,12.448,12.396,12.307,12.199,12.007,11.923,11.836,11.699
2,BUSHIDG_COAST,2016-01-03,11.457,11.381,11.332,11.291,11.271,11.266,11.235,11.241,...,12.358,12.314,12.241,12.175,12.075,11.953,11.746,11.651,11.558,11.416
3,BUSHIDG_COAST,2016-01-04,11.358,11.280,11.228,11.182,11.162,11.150,11.111,11.100,...,12.666,12.581,12.395,12.277,12.145,11.996,11.804,11.699,11.598,11.450
4,BUSHIDG_COAST,2016-01-05,11.492,11.418,11.373,11.338,11.336,11.334,11.300,11.296,...,12.781,12.711,12.570,12.472,12.350,12.210,12.008,11.906,11.806,11.658


In [10]:
def plot_load_by_date(df = sorted_load, station = 'BUSHIDG_COAST', date = '1/01/2016'):
    date = pd.to_datetime(date)
    try:
        single_profile = df[(df['Station']==station) & (df['Date'] == date)]
        print(single_profile)

        fig = go.Figure()
        fig.add_trace(go.Scatter(x = load_columns, y = single_profile.iloc[0], name = station))
        date_str = date.strftime('%m/%d/%Y')
        fig.update_layout(title = f"Daily Load - {date_str} associated with {station}", showlegend=True)
        fig.update_xaxes(tickangle=45, title_text="Time of Day")
        fig.update_yaxes(title_text="Load (kW)")
        fig.show()
    except:
        print("Exception: There does not appear to be data for this selection.")
    
plot_load_by_date()

         Station       Date  00:00:00  00:15:00  00:30:00  00:45:00  01:00:00  \
0  BUSHIDG_COAST 2016-01-01    11.489    11.413    11.364    11.325    11.312   

   01:15:00  01:30:00  01:45:00  ...  21:30:00  21:45:00  22:00:00  22:15:00  \
0    11.307    11.273    11.273  ...    12.304    12.261    12.177    12.112   

   22:30:00  22:45:00  23:00:00  23:15:00  23:30:00  23:45:00  
0    12.015      11.9    11.711    11.624    11.536    11.399  

[1 rows x 98 columns]


In [11]:
batch_size = 4
input_size = int(len(load_columns)/batch_size)

In [12]:

'''Referenced from https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files'''
class LoadDataset(Dataset):
    def __init__(self, df):
        self.df = df


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx][2:].astype(np.float32).values.reshape(batch_size,24)
        try:
            labels = self.df.iloc[idx+1][2:6].astype(np.float32).values
        except:
            labels = self.df.iloc[idx][2:6].astype(np.float32).values
        sample = torch.from_numpy(sample)
        labels = torch.from_numpy(labels)
        return sample, labels
    

    

#Initiate LoadDataset, check target function
Load = LoadDataset(sorted_load)


# Train, Test Split

In [13]:
train_df, test_df = train_test_split(sorted_load, test_size=0.2, shuffle=False)
train_dataset = LoadDataset(train_df)
test_dataset = LoadDataset(test_df)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size,  shuffle=False)

# Define and train Neural Network Model

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MultiVarLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_targets):
        super(MultiVarLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, num_targets)
        self.hidden_list = []
        
    def forward(self, x):
        x.to(device)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        self.hidden_list.append([self.num_layers,x.size(0), self.hidden_size, h0])
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
device

device(type='cpu')

In [15]:
load_lstm_model = MultiVarLSTM(input_size=24, hidden_size=100, num_layers=2, num_targets=4)
optimizer = torch.optim.Adam(load_lstm_model.parameters(), lr=0.01)
loss_function = nn.MSELoss()

In [16]:
epochs = 2
min_validation_loss = np.inf
load_lstm_model.train() # set model to training mode
# if a GPU is available, use it
# if a model is in the directory, load it instead of training
"""Also referenced from https://www.geeksforgeeks.org/training-neural-networks-with-validation-using-pytorch/"""

for epoch in range(epochs):
    training_loss = 0.0
    print("Training Epoch: {}".format(epoch+1))
    for data, labels in tqdm(train_dataloader):
        if torch.cuda.is_available():
            data, labels = data.to(device), labels.to(device) # move to GPU, if available
        optimizer.zero_grad()
        target = load_lstm_model(data)
        loss = loss_function(target, labels)
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(load_lstm_model.parameters(), 100)
        optimizer.step() # update the weights
        training_loss += loss.item()
    validation_loss = 0.0
    print("Validating Epoch: {}".format(epoch+1))
    load_lstm_model.eval()
    for data, labels in test_dataloader:
        if torch.cuda.is_available():
            data, labels = data.to(device), labels.to(device) # move to GPU, if available
        target = load_lstm_model(data)
        loss = loss_function(target, labels)
        validation_loss += loss.item()*data.size(0)
    print(f'Epoch {epoch+1}/{epochs} Loss: {training_loss/len(train_dataloader)}')
    if min_validation_loss > validation_loss:
        print(f'Validation Loss Decreased({min_validation_loss:.6f}--->{validation_loss:.6f})\t Saving The Model')


Training Epoch: 1


100%|██████████| 15128/15128 [01:17<00:00, 194.63it/s]


Validating Epoch: 1
Epoch 1/2 Loss: 811.1929298388279
Validation Loss Decreased(inf--->280.189960)	 Saving The Model
Training Epoch: 2


100%|██████████| 15128/15128 [01:20<00:00, 188.59it/s]


Validating Epoch: 2
Epoch 2/2 Loss: 8284.940800324608
Validation Loss Decreased(inf--->537.694903)	 Saving The Model


In [17]:
def evaluate_lstm_accuracy():
    """
    Return the error between the LSTM predictions and the actual values.
    """
    load_lstm_model.eval()
    test_loss = 0.0
    for data, labels in test_dataloader:
        if torch.cuda.is_available():
            data, labels = data.to(device), labels.to(device) # move to GPU, if available
        target = load_lstm_model(data)
        loss = loss_function(target, labels)
        test_loss += loss.item()*data.size(0)
    #test_loss /= len(test_dataloader.dataset) # average loss
    print(f'Test Loss: {test_loss}')
    return test_loss
evaluate_lstm_accuracy()


Test Loss: 537.6949031874537


537.6949031874537

In [24]:
def plot_lstm_predictions(profile_idx = 0):
    """
    Plot LSTM model predictions and the actual loads in Plotly Express.
    """
    load_lstm_model.eval()
    data, labels1 = next(iter(train_dataloader))
    target = load_lstm_model(data).detach().cpu().numpy().flatten()[:4]
        #print(target)
        # print(load_lstm_model(data).size())

    load_plus_one = load_columns + ['t1','t2','t3','t4']
    # print(sorted_load.iloc[0])
    original_load_wForecast = pd.concat([sorted_load.iloc[profile_idx][2:], pd.Series(target)], axis=0) 
    fig = px.line(x=load_plus_one, y=original_load_wForecast, title='Loads with Forecast')
    fig.add_vrect(x0=len(load_columns), x1=len(load_columns)+4, y0=0, y1=1)
    fig.update_xaxes(tickangle=45, title_text="Time of Day")
    fig.update_layout(xaxis_title='Time', yaxis_title='Load (kW)')

   # fig = px.line(x=load_columns, y=target.detach().cpu().numpy(), title='Predicted Loads')
#
    fig.show()
plot_lstm_predictions(75000)

RuntimeError: input must have 3 dimensions, got 2

# Show the forecast with the next actual values

# Akaike Information Criterion (AIC), out-of-sample error estimate, lower AIC values indicate higher fidelity for out of sample predictions by moving average.
# - Neural Network and Moving Average Baseline

In [19]:
#print(nn_predictions)
#print(val_df)


# nn_predictions = pd.DataFrame(nn_predictions)
# #nn_predictions.plot()
# nn_mse =  np.mean((nn_predictions - val_df)**2)
# moving_average_mse = ((vdma_df-vdf)**2).mean()
# print(nn_mse)
# print(moving_average_mse)
# def aic(mse, k):
#     return 2 * k - 2 * mse
# aic_nn_mse = aic(nn_mse, k=19058)
# aic_vdma_mse = aic(moving_average_mse, k=3)

# print("NEURAL NETWORK AIC")
# print(aic_nn_mse)
# print("MOVING AVERAGE AIC")
# print(aic_vdma_mse)


#